In [1]:
from alive_progress import alive_it
from pathlib import Path
from pytube import YouTube, Playlist
from urllib.parse import urlparse, parse_qs
import shutil

In [2]:
a = 'https://youtu.be/gsEs6_JIocA?si=Fe66SsuBkUEivni3'
b = 'https://www.youtube.com/watch?v=gsEs6_JIocA'
c = 'https://www.youtube.com/playlist?list=PLBn0C5qlKMEz6qdA7gr6vAPVnhmomdBYt'
d = 'https://www.youtube.com/watch?v=gsEs6_JIocA&list=PLBn0C5qlKMEz6qdA7gr6vAPVnhmomdBYt&index=1'
e = 'https://youtube.com/playlist?list=PLBn0C5qlKMEz6qdA7gr6vAPVnhmomdBYt&si=MyonDtZ_vtDHnHwk'

In [4]:
def get_url_list():
    url = input('Paste link here: ')
    while True:
        result = urlparse(url)
        netloc = result.netloc
        if 'youtu.be' == netloc:
            url_list = [url]
            return url_list
        elif 'youtube.com' in netloc:
            query = result.query
            params = parse_qs(query)
            keys = list(params.keys())
            if 'list' in keys:
                p = Playlist(url)
                url_list = list(p.video_urls)
                return url_list
            elif 'v' in keys:
                url_list = [url]
                return url_list
            else:
                url = input('Invalid URL. Please try again: ')
        else:
            url = input('Invalid URL. Please try again: ')

In [44]:
def run():
    output_path = Path.cwd() / 'output'
    if not output_path.is_dir():
        output_path.mkdir()
    
    file_list = list(output_path.iterdir())
    if file_list:
        shutil.rmtree(output_path)
        output_path.mkdir()
    
    options = {
        'length': 70,
        'spinner': 'classic',
        'bar': 'classic2',
        'receipt_text': True,
        'dual_line': True
    }
    
    url_list = get_url_list()
    results = alive_it(
        url_list,
        len(url_list),
        finalize=lambda bar: bar.text('Renaming TV subtitles: done'),
        **options
    )
    
    for url in results:
        results.text(': ')
        yt = YouTube(url)
        yt.streams \
        .filter(progressive=True) \
        .first() \
        .download(output_path)
